In [1]:
!pip install scrapy 


     |████████████████████████████████| 235kB 4.9MB/s 
     |████████████████████████████████| 61kB 22.4MB/s 
     |████████████████████████████████| 3.1MB 45.4MB/s 
     |████████████████████████████████| 2.3MB 31.1MB/s 
     |████████████████████████████████| 174kB 44.4MB/s 
     |████████████████████████████████| 61kB 26.3MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=fcb7778f049d14baaadf26f67fa0cc306d6fed469e7eb0172d0a9758551ae5a7
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher


In [0]:
import logging
import os
import pandas as pd
import re
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors.lxmlhtml import LxmlLinkExtractor
from googlesearch import search
logging.getLogger('scrapy').propagate = False

In [0]:
def get_urls(tag, n, language):
    urls = [url for url in search(tag, stop=n, lang=language)][:n]
    return urls

In [0]:
html_text=get_urls('doctors in india justdial', 100 , 'en')

In [5]:
html_text

['https://www.justdial.com/Delhi/57/Doctor_fil',
 'https://www.justdial.com/Delhi/General-Physician-Doctors/nct-10892680',
 'https://www.justdial.com/Delhi/On-Call-Doctor/nct-11310787',
 'https://www.justdial.com/Mumbai/General-Physician-Doctors/nct-10892680',
 'https://www.justdial.com/Pune/General-Physician-Doctors/nct-10892680',
 'https://www.justdial.com/Mumbai/Online-Doctors/nct-10961244',
 'https://www.justdial.com/Delhi/Clinics/nct-10101647',
 'https://www.justdial.com/Delhi/Allopathic-Doctors/nct-10994509',
 'https://www.justdial.com/Mumbai/Allopathic-Doctors/nct-10994509',
 'https://www.justdial.com/Delhi/General-Surgeon-Doctors/nct-10551083',
 'https://www.justdial.com/Delhi/Doctors-For-Height/nct-11112947',
 'https://www.justdial.com/Jaipur/Doctors-Medicine/nct-10167598',
 'https://www.justdial.com/Mumbai/ENT-Doctors/nct-10189442',
 'https://www.justdial.com/Delhi/Ayurvedic-Doctors/nct-10029616',
 'https://m.economictimes.com/markets/stocks/recos/sell-just-dial-target-rs-562

In [0]:
class MailSpider(scrapy.Spider):
    
    name = 'email'
    
    def parse(self, response):
        
        links = LxmlLinkExtractor(allow=()).extract_links(response)
        links = [str(link.url) for link in links]
        links.append(str(response.url))
        
        for link in links:
            yield scrapy.Request(url=link, callback=self.parse_link) 
            
    def parse_link(self, response):
        
        for word in self.reject:
            if word in str(response.url):
                return
            
        html_text = str(response.text)
        mail_list = re.findall('\w+@\w+\.{1}\w+', html_text)

        dic = {'email': mail_list, 'link': str(response.url)}
        df = pd.DataFrame(dic)
        
        df.to_csv(self.path, mode='a', header=False)
        df.to_csv(self.path, mode='a', header=False)

In [0]:
def ask_user(question):
    response = input(question + ' y/n' + '\n')
    if response == 'y':
        return True
    else:
        return False
def create_file(path):
    response = False
    if os.path.exists(path):
        response = ask_user('File already exists, replace?')
        if response == False: return 
    
    with open(path, 'wb') as file: 
        file.close()

In [0]:
def get_info(tag, n, language, path, reject=[]):
    
    create_file(path)
    df = pd.DataFrame(columns=['email', 'link'], index=[0])
    df.to_csv(path, mode='w', header=True)
    
    print('Collecting Google urls...')
    google_urls = get_urls(tag, n, language)
    
    print('Searching for emails...')
    process = CrawlerProcess({'USER_AGENT': 'Mozilla/5.0'})
    process.crawl(MailSpider, start_urls=google_urls, path=path, reject=reject)
    process.start()
    
    print('Cleaning emails...')
    df = pd.read_csv(path, index_col=0)
    df.columns = ['email', 'link']
    df = df.drop_duplicates(subset='email')
    df = df.reset_index(drop=True)
    df.to_csv(path, mode='w', header=True)
    
    return df

In [0]:
bad_words = ['facebook', 'instagram', 'youtube', 'twitter', 'wiki']

In [10]:
df = get_info('gyms kolkata justdial', 5 , 'en', 'doctor_lead.csv', reject=bad_words)

Searching for emails...
Cleaning emails...


In [11]:
print(df)

                    email                                               link
0                     NaN                                                NaN
1        jobsprite@2x.png                      https://www.justdial.com/Jobs
2       monsterjob@2x.png                      https://www.justdial.com/Jobs
3     rusers@justdial.com  https://www.justdial.com/travel/international-...
4             abc@xyz.com  https://www.justdial.com/Kolkata/Fitness-Centr...
5   feedback@justdial.com              https://www.justdial.com/Free-Listing
6   security@justdial.com   https://wap.justdial.com/reportabug.php?source=7
7   support@thomascook.in   https://www.justdial.com/Travel/Foreign-Exchange
8           vinit@fnp.com               https://www.justdial.com/order-books
9  vinitpatel03@yahoo.com               https://www.justdial.com/order-books
